In [1]:
# 準備工作

from bs4 import BeautifulSoup
import requests
# import bs4
from selenium import webdriver
import time
import datetime
import re
import sqlite3
import os

db_name = "../data/21_century.sqlite"
db_url_list = "../data/21_century_URLs.sqlite"

In [ ]:
# 建立紀錄已經看過的內頁url的資料庫
# 如果這個資料庫已經被建立過了 就略過

# url: 內網url中的ID 具primary key性質
# modDate: 檢視內頁的日期跟時間

if os.path.isfile(db_url_list) == False:

    with sqlite3.connect(db_url_list) as conn_id_list:
        
        c = conn_id_list.cursor()
        
        c.execute("""CREATE TABLE urlList(
        url text unique not null,
        modDate datetime 
        )""")

    conn_id_list.close()

In [2]:
# 建立儲存內頁url跟內頁內容用的資料庫
# 如果這個資料庫已經被建立過了 就略過

# url: 內網url 具primary key性質
# soup: 內頁內容
# nosql: 用於紀錄資料紀錄於nosql資料庫的狀態
# rdb: 用於紀錄資料紀錄於sql資料庫的狀態
# getTime: 紀錄內頁的日期跟時間

if os.path.isfile(db_name) == False:

    with sqlite3.connect(db_name) as conn:
        
        c = conn.cursor()
        
        c.execute("""CREATE TABLE rental(
        url text unique not null,
        soup text not null,
        nosql text,
        rdb text,
        getTime datetime 
        )""")
        
    conn.close()

In [ ]:
# 定義 insert_urls() 方法

# 將爬下來的ID存到爬過的url紀錄中

# 參數: 
# db_name: 要塞的資料庫名稱(sqlite檔案)
# id_list: 含有內頁ID的list

def insert_urls(db_url_list, id_list):
    
    with sqlite3.connect(db_url_list) as conn_insert_url:
               
        c_iu = conn_insert_url.cursor()

        print(id_list)
        
        for each_id in id_list:
        
            try:
        
                today = datetime.datetime.today()
                today = today.strftime('%Y-%m-%d %H:%M:%S')

                values = "'http://www.century21.com.tw/index/Rental/RentPage/" + str(each_id) + "','" + str(today) + "'"       

                insertString = "INSERT INTO urlList (url, modDate) VALUES(" + values + ");"
                c_iu.execute(insertString)
            
            except Exception as e:
                
                print("例外: " + str(e))
                print(each_id)
            
    c_iu.close()

In [ ]:
# 找出所有已經被爬過的內頁url

with sqlite3.connect(db_name) as retr_url:
        
    c = retr_url.cursor()

    c.execute("SELECT url from rental")
    url_completed = c.fetchall()
    
retr_url.close()
# url_completed[0][0][-6:]

In [ ]:
# 定義 crawl_links_single() 方法
# 從一個外網頁面找出該頁面裡所有的內頁ID

def crawl_links_single(driver):

    soup = BeautifulSoup(driver.page_source,'html.parser')
    soup_hrefs = soup.select('div[class="main clearfix"] > div > a')
    hrefs = []

    for each in soup_hrefs:
        hrefs.append(each['href'].split('/')[-1])
#         hrefs.append(each['href'])

    return hrefs

In [ ]:
# 定義 crawl_n_insert() 方法
# 爬內頁然後儲存內容到資料庫中

def crawl_n_insert(url, db_name):
    
#     driver = webdriver.PhantomJS()
#     driver.get(url)
#     time.sleep(2.5)
#     page_source = str(driver.page_source)
    page_source = requests.get(url)
    page_source = str(page_source.text)
    
    # 處理引號
    
    page_source = page_source.replace("\"","\"\"")
    page_source = page_source.replace('\'',"\'\'")
    
    with sqlite3.connect(db_name) as conn_insert_page:
               
        c_ip = conn_insert_page.cursor()
        
        try:

            today = datetime.datetime.today()
            today = today.strftime('%Y-%m-%d %H:%M:%S')

            values = "\"" + str(url) + "\",'" + page_source + "','','','" + str(today) + "'"

#             print(values)

            insertString = "INSERT INTO rental (url, soup, nosql, rdb, getTime) VALUES(" + values + ");"

#             print(insertString)

            c_ip.execute(insertString)

        except Exception as e:

            print(url)
            print("例外: " + str(e))
        finally:
            c_ip.close()

In [ ]:
# 主程序1:
# 從外網把所有的內頁url爬下來

main_url = "http://www.century21.com.tw/index/Rental/Rent"

end_loop = False
driver = webdriver.PhantomJS()
driver.get(main_url)
time.sleep(2.5)

while not end_loop:
    
    try:
    
        print(driver.current_url)
        id_list = crawl_links_single(driver)
        return_count = len(id_list)

        if return_count > 0:
            insert_urls(db_url_list, id_list)
    
        driver.find_element_by_link_text("下一頁").click()
        time.sleep(1)
        
    except Exception as e:
        
        print("例外: " + str(driver.current_url) + " \r; " + str(e))
        end_loop = True
        
print("內頁url搜尋結束")

In [ ]:
# 檢查 db_url_list 結果

with sqlite3.connect(db_url_list) as conn_url_list:
    
    c_ul = conn_url_list.cursor()
    qryString = "SELECT distinct count(*) from urlList limit 1;"
    c_ul.execute(qryString)
    results = c_ul.fetchall()

c_ul.close()

print((results))

In [ ]:
# 主程序2:
# 從外網把所有的內頁內容爬下來

# 取得所有的url

with sqlite3.connect(db_url_list) as conn_call_list:
    
    c_cl = conn_url_list.cursor()
    qryString = "SELECT url from urlList;"
    c_cl.execute(qryString)
    results = c_cl.fetchall()

c_cl.close()

# 略過已經做過的網址
        
for each in results:
    try:
        print(each[0])
        crawl_n_insert(each[0],db_name)
    except Exception as e:
        print(e)

print("所有的內頁內容都已爬過且儲存到資料庫中")

In [ ]:
# 檢查 db_name 結果

with sqlite3.connect(db_name) as conn_db_name:
    
    c_dn = conn_db_name.cursor()
    qryString = "SELECT count(url) from rental;"
    c_dn.execute(qryString)
    results = c_dn.fetchall()

c_dn.close()
    
print((results))